# Scenario Report

In [ ]:
# Copyright (c) 2021 *****************. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

#%reload_ext autoreload
#%autoreload 2

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np
from switch_model.reporting.report_functions import *

# TODO: remove input folder = '../../MODEL_RUNS/test_PCE/inputs/hourly_90'

#get the name of the current directory to specify the scenario name and identify the output directory
scenario_name = str(Path.cwd()).split('\\')[-1]
if scenario_name == 'inputs':
    data_dir = Path.cwd() / '../outputs/'
    inputs_dir = Path.cwd() / '../inputs/'
    scenario_name = 'N/A'
else:
    data_dir = Path.cwd()/ f'../../outputs/{scenario_name}/'
    inputs_dir = Path.cwd() / f'../../inputs/{scenario_name}/'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format

#allow the notebook to display plots in html report
###################################################
plotly.offline.init_notebook_mode()

print(f'Scenario Name: {scenario_name}')

In [ ]:
# load data from csvs
costs_by_gen = pd.read_csv(data_dir / 'costs_by_gen.csv')
costs_by_tp = pd.read_csv(data_dir / 'costs_by_tp.csv')
dispatch = pd.read_csv(data_dir / 'dispatch.csv')
emissions = pd.read_csv(data_dir / 'emissions.csv')
with open(inputs_dir / 'ghg_emissions_unit.txt', 'r') as unit:
    emissions_unit = unit.read()
fixed_costs = pd.read_csv(inputs_dir / 'fixed_costs.csv')
gen_build_predetermined = pd.read_csv(inputs_dir / 'gen_build_predetermined.csv', usecols=['GENERATION_PROJECT'])
gen_cap = pd.read_csv(data_dir / 'gen_cap.csv')
generation_projects_info = pd.read_csv(inputs_dir / 'generation_projects_info.csv')
grid_emissions = pd.read_csv(inputs_dir / 'grid_emissions.csv')
hybrid_pair = hybrid_pair_dict(generation_projects_info)
load_balance = pd.read_csv(data_dir / 'load_balance.csv')
nodal_prices = pd.read_csv(inputs_dir / 'nodal_prices.csv')
RA_summary = pd.read_csv(data_dir / 'RA_summary.csv')
rec_value = pd.read_csv(inputs_dir / 'rec_value.csv').loc[0,'rec_resale_value'].item()
storage_builds = pd.read_csv(data_dir / 'storage_builds.csv')
storage_cycle_count = pd.read_csv(data_dir / 'storage_cycle_count.csv')
storage_dispatch = pd.read_csv(data_dir / 'storage_dispatch.csv')
system_power = pd.read_csv(data_dir / 'system_power.csv')
timestamps = pd.read_csv(inputs_dir / 'timepoints.csv', parse_dates=['timestamp'], usecols=['timepoint_id','timestamp'])


technology_color_map = {
 'Small Hydro':'Purple',
 'Consumed Small Hydro':'Purple',
 'Excess Small Hydro':'Plum',
 'Onshore Wind':'DeepSkyBlue',
 'Consumed Onshore Wind':'DeepSkyBlue',
 'Excess Onshore Wind':'LightSkyBlue',
 'Offshore Wind':'Navy',
 'Consumed Offshore Wind':'Navy',
 'Excess Offshore Wind':'MediumSlateBlue',
 'Solar PV':'Gold',
 'Hybrid Solar PV':'Gold',
 'Consumed Solar PV':'Gold',
 'Excess Solar PV':'Yellow',
 'CSP':'Orange',
 'Geothermal':'Sienna',
 'Consumed Geothermal':'Sienna',
 'Storage':'Green',
 'Storage Discharge':'Green',
 'Hybrid Storage':'Green',
 'Grid Energy':'Red',
 '(?)':'Black'}

# Renewable Energy Goal

## Portfolio Renewable Percentage

In [ ]:
  
print(f'Time-coincident renewable percentage: {format_percent(hourly_renewable_percentage(load_balance))}')
print(f'Annual volumetric renewable percentage: {format_percent(annual_renewable_percentage(load_balance))}')

In [ ]:
# Add hourly renewable heatmap

### Carbon Emissions

In [ ]:
print(f'Total Annual Emissions: {emissions["Emission Rate"].sum().round(1)} {emissions_unit.split("/")[0]}')
print(f'Delivered Emission Factor: {emissions["Delivered Emission Factor"].mean().round(3)} {emissions_unit}')

In [ ]:
build_hourly_emissions_heatmap(grid_emissions, emissions, emissions_unit).show()

# Portfolio

## Generator Portfolio
The sunburst chart describes the built portfolio at various levels of detail, which shows how the outer rings relate to the inner rings
- Inner circle: contract status (contracted or additional project)
- Middle ring: technology type (e.g. solar, wind, ...)
- Outer ring: specific project name

For example, individual projects in the outer ring belong to a specific technology type in the middle ring, which can either be part of the existing/contracted portfolio, or the additional portfolio.


In [ ]:
portfolio = generator_portfolio(gen_cap, gen_build_predetermined)

portfolio_sunburst = px.sunburst(portfolio, path=['Status','Technology','generation_project'], values='MW', color='Technology', color_discrete_map=technology_color_map,
 width=1000, height=1000,
 title='Energy Portfolio by Project Name, Technology Type, and Contract Status (MW)')
portfolio_sunburst.update_traces(textinfo='label+value')
portfolio_sunburst.show()

## Generation Cost by Project
This should show contract costs, Nodal costs, and storage revenues

In [ ]:
gen_costs = generator_costs(costs_by_gen, storage_dispatch, hybrid_pair, gen_cap)

generator_costs_melted = gen_costs.drop(columns='Total Cost').melt(id_vars='generation_project', var_name='Cost', value_name='$/MWh')

generator_cost_fig = px.bar(generator_costs_melted, 
                            title='Average Generator Cost per MWh Generated', 
                            x='generation_project', 
                            y='$/MWh', 
                            color='Cost', 
                            color_discrete_map={'Energy Contract Cost': 'Green',
                                                'Capacity Contract Cost': 'Orange',
                                                'Delivery Cost': 'lightblue',
                                                'Pnode Revenue':'Red', 
                                                'Storage Arbitrage Revenue':'Purple'}).update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='black')

generator_cost_fig.add_scatter(x=gen_costs.generation_project, y=gen_costs['Total Cost'], mode='markers+text', text=gen_costs['Total Cost'], textposition='top center', line=dict(color='black', width=1), name='Total Cost')

generator_cost_fig.show()



### Reduced Costs of non-selected generators

## Power Content Label


In [ ]:
power_content = power_content_label(load_balance, dispatch, generation_projects_info)

delivered_energy_pie = px.pie(power_content, values='MWh', names='Source', title='Power Content of Delivered Energy (MWh)', color='Source', color_discrete_map=technology_color_map,
width=600, height=600
 )
delivered_energy_pie.update_traces(textinfo='percent+label+value')
delivered_energy_pie.show()

# Costs

## Delivered Electricity Cost Summary

### Hourly average cost of power

In [ ]:
hourly_costs = hourly_cost_of_power(system_power, costs_by_tp, RA_summary, gen_cap, storage_dispatch, fixed_costs, rec_value, load_balance)
hourly_cost_plot = build_hourly_cost_plot(hourly_costs, load_balance)
hourly_cost_plot.show()

### Cost Breakdown

In [ ]:
cost_table, resale_table = construct_cost_and_resale_tables(hourly_costs, load_balance, RA_summary)
display(cost_table.set_index('Cost Component'))


### Resale Values

In [ ]:
display(resale_table.set_index('Cost Component'))

### Curtailment summary

## Resource Adequacy Open Position

In [ ]:
build_ra_open_position_plot(RA_summary).show()

# Portfolio Dispatch

In [ ]:
build_dispatch_plot(generation_projects_info, dispatch, storage_dispatch, load_balance, system_power, technology_color_map).show()

### Wholesale prices

In [ ]:
build_nodal_prices_plot(nodal_prices, timestamps, generation_projects_info).show()

### Battery state of charge

In [ ]:
build_state_of_charge_plot(storage_dispatch, storage_builds, generation_projects_info).show()

### Binding Demand Constraints

### Month hour average dispatch

In [ ]:
# under construction

### Month-hour average net position

Positive values represent excess generation  
Negative values represent an open position

In [ ]:
build_open_position_plot(load_balance).show()

## Battery Cycling and State of Charge Stats

In [ ]:
display(construct_storage_stats_table(storage_cycle_count, storage_builds, storage_dispatch))

# Impact Metrics
- Duck curve impacts (curtailment, ramping, etc)
- Avoided Emissions

# Assumptions

In [ ]:
pd.set_option('display.max_rows',100)
gen_assumptions = pd.read_csv(inputs_dir / 'generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','ppa_energy_cost','gen_capacity_limit_mw'])
gen_assumptions = gen_assumptions[gen_assumptions['gen_energy_source'] != 'Electricity']
gen_assumptions = gen_assumptions.sort_values(by='GENERATION_PROJECT')
gen_assumptions = gen_assumptions.set_index('GENERATION_PROJECT')
gen_assumptions

# Storage Assumptions

In [ ]:
storage_assumptions = pd.read_csv(inputs_dir / 'generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','ppa_capacity_cost','gen_capacity_limit_mw', 'storage_roundtrip_efficiency','storage_charge_to_discharge_ratio','storage_energy_to_power_ratio','storage_leakage_loss','storage_hybrid_generation_project','storage_hybrid_min_capacity_ratio','storage_hybrid_max_capacity_ratio'])
#change capacity cost to $/kw-mo
storage_assumptions['ppa_capacity_cost'] = storage_assumptions['ppa_capacity_cost'] / 12000

storage_assumptions = storage_assumptions.rename(columns={
    'storage_roundtrip_efficiency':'RTE',
    'storage_charge_to_discharge_ratio':'charge/discharge_ratio',
    'storage_energy_to_power_ratio':'storage_hours',
    'storage_leakage_loss':'soc_leakage_loss',
    'storage_hybrid_generation_project':'paired_hybrid_gen',
    'storage_hybrid_min_capacity_ratio':'hybrid_min_capacity_ratio',
    'storage_hybrid_max_capacity_ratio':'hybrid_max_capacity_ratio',
})

storage_assumptions = storage_assumptions[storage_assumptions['gen_energy_source'] == 'Electricity']
storage_assumptions = storage_assumptions.sort_values(by='GENERATION_PROJECT')
storage_assumptions = storage_assumptions.set_index('GENERATION_PROJECT')
storage_assumptions = storage_assumptions[['gen_tech','ppa_capacity_cost','gen_capacity_limit_mw', 'RTE','storage_hours','charge/discharge_ratio','soc_leakage_loss','paired_hybrid_gen','hybrid_min_capacity_ratio','hybrid_max_capacity_ratio']]
storage_assumptions

In [ ]:
# export scenario summary